In [85]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime


def get_match_data(start_date, time_now_formatted, devengine):
    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )    
    
    # Get historical match data on clay surface between start date and yesterday
    elo_all = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_All where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get today's matches on clay surface that haven't yet been resulted
    elo_data_all = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_All where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,
    )

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay, elo_grass, elo_data_grass, elo_all, elo_data_all


# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today = time_now
two_years_ago = (today - datetime.timedelta(days=365 * 2)).strftime("%Y-%m-%d")

(
    elo_hard,
    elo_clay,
    elo_data_hard,
    elo_data_clay,
    elo_grass,
    elo_data_grass,
    elo_all,
    elo_data_all,    
) = get_match_data(two_years_ago, time_now_formatted, devengine)


def get_player_record(player, opponent_rank, history, range_low, range_high, auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(
            row.Fav, row.Dog_Rank, elo, low_limit, high_limit, True
        )
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(
                row.Fav, row.Dog_Rank, elo, low_limit, high_limit, True
            )

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(
            row.Dog, row.Fav_Rank, elo, low_limit, high_limit, True
        )
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(
                row.Dog, row.Fav_Rank, elo, low_limit, high_limit, True
            )

        # New code to calculate player's record against rank 0 to 100
        fav_record, _ = get_player_record(
            row.Fav, 100, elo, 0, 100, False
        )
        dog_record, _ = get_player_record(
            row.Dog, 100, elo, 0, 100, False
        )
        
        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                    "Resulted": [row.Resulted],
                    "fav_rank": [row.Fav_Rank],
                    "dog_rank": [row.Dog_Rank],
                    "Elo_Fav_Elo": [row.Elo_Fav_Elo],
                    "Elo_Dog_Elo": [row.Elo_Dog_Elo],
                    "Fav_Top100": [round(fav_record,1)],  # New column
                    "Dog_Top100": [round(dog_record,1)]  # New column
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df


results_hard = get_filtered_data(elo_data_hard, elo_hard)
results_clay = get_filtered_data(elo_data_clay, elo_clay)
results_grass = get_filtered_data(elo_data_grass, elo_grass)
#results_grass = get_filtered_data(elo_data_grass, elo_hard)
results_all = get_filtered_data(elo_data_all, elo_all)


def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv("mensserving.csv")

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv("womensserving.csv")

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        # serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(
                result_df, serving, how="left", left_on=["Fav"], right_on=["Name"]
            )
            result = pd.merge(
                result, serving, how="left", left_on=["Dog"], right_on=["Name"]
            )
            result.rename(
                columns={
                    "Service Games Won_x": "Fav_Serve%",
                    "Service Games Won_y": "Dog_Serve%",
                    "Return Games Won_x": "Fav_Return%",
                    "Return Games Won_y": "Dog_Return%",
                },
                inplace=True,
            )
            result.drop(columns=["Name_x", "Name_y"], inplace=True)

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None

    return result, result_serving


_, serving_hard = process_serving_data(results_hard)
_, serving_clay = process_serving_data(results_clay)
_, serving_grass = process_serving_data(results_grass)
_, serving_all = process_serving_data(results_all)


data_concat = pd.DataFrame(columns=["Date", "Player", "Odds", "Win/Loss"])
for dataset_type in [("Winner", "Win"), ("Loser", "Loss")]:
    df = elo_hard[["Date", dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df["Player"] = df[dataset_type[0]]
    df["Odds"] = df[f"{dataset_type[0]}_Odds"]
    df["Win/Loss"] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat["Odds"] = data_concat.Odds.astype(float)


def analyse_matchups(result_df, data_concat):
    for _, matchup in result_df.iterrows():
        player1 = matchup.Fav
        player2 = matchup.Dog
        player1_odds = float(matchup.Fav_Odds)
        player1_odds_hi = player1_odds + 0.15
        player1_odds_lo = player1_odds - 0.15
        player2_odds = float(matchup.Dog_Odds)
        player2_odds_hi = player2_odds + 0.15
        player2_odds_lo = player2_odds - 0.15
        player1 = data_concat[data_concat["Player"] == player1].copy()
        player2 = data_concat[data_concat["Player"] == player2].copy()
        player2 = player2[
            (player2["Odds"] > player2_odds_lo) & (player2["Odds"] < player2_odds_hi)
        ]
        if len(player2) > 0:
            winperc2 = len(player2[player2["Win/Loss"] == "Win"]) / len(player2)
        else:
            winperc2 = 0
        player1 = player1[
            (player1["Odds"] > player1_odds_lo) & (player1["Odds"] < player1_odds_hi)
        ]
        if len(player1) > 0:
            winperc1 = len(player1[player1["Win/Loss"] == "Win"]) / len(player1)
        else:
            winperc1 = 0
        if len(player1) > 5 and len(player2) > 3:
            print(
                matchup.Time,
                f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",
                f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})",
            )
            print(len(player1), winperc1, len(player2), winperc2)


# analyse_matchups(serving_clay,data_concat)
final_hard = serving_hard
final_clay = serving_clay
final_grass = serving_grass
final_all = serving_all

Error occured while merging the dataframes.
Error: 'Fav'


In [86]:
def last_five(df,pastmatches):
    for index, row in df.iterrows():
        fav=row.Fav
        dog=row.Dog
        last_five_matches_fav=pastmatches[(pastmatches['Winner']==fav)|(pastmatches['Loser']==fav)].tail(5)
        if len(last_five_matches_fav)>0:  
            fav_last_five_win_perc=len(last_five_matches_fav[last_five_matches_fav['Winner']==fav])/len(last_five_matches_fav)
        else:
            fav_last_five_win_perc=0
        last_five_matches_dog=pastmatches[(pastmatches['Winner']==dog)|(pastmatches['Loser']==dog)].tail(5)
        if len(last_five_matches_dog)>0:    
            dog_last_five_win_perc=len(last_five_matches_dog[last_five_matches_dog['Winner']==dog])/len(last_five_matches_dog) 
        else:
            dog_last_five_win_perc=0
        df.at[index, 'fav_last_five_win_perc'] = fav_last_five_win_perc
        df.at[index, 'dog_last_five_win_perc'] = dog_last_five_win_perc
    return df

if final_hard is not None:
    final_hard=last_five(final_hard,elo_hard)
    final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
    final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)    

In [87]:
if final_clay is not None:
    final_clay=last_five(final_clay,elo_clay)
    final_clay['Dog_Odds']=final_clay['Dog_Odds'].astype(float)
    final_clay['Fav_Odds']=final_clay['Fav_Odds'].astype(float)

In [88]:
if final_grass is not None:
    final_grass=last_five(final_grass,elo_grass)
    final_grass['Fav_Odds']=final_grass['Fav_Odds'].astype(float)
    final_grass['Dog_Odds']=final_grass['Dog_Odds'].astype(float)  
final_all=last_five(final_all,elo_all)
final_all['Fav_Odds']=final_all['Fav_Odds'].astype(float)
final_all['Dog_Odds']=final_all['Dog_Odds'].astype(float)        

In [89]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Daniel Cukierman beat Alexey Vatutin


In [90]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

David Jorda Sanchis beat Giovanni Fonio
Chun Hsin Tseng beat Louis Wessels


In [91]:
#for _,i in elo_data_grass.iterrows():
#    check1=elo_grass[((elo_grass['Winner']==i.Winner)&(elo_grass['Loser']==i.Loser))|((elo_grass['Loser']==i.Winner)&(elo_grass['Winner']==i.Loser))]
#    if check1.empty==False:
#        for _, x in check1.iterrows():
#            print(f"{x.Winner} beat {x.Loser}")

In [92]:
for _,i in elo_data_all.iterrows():
    check1=elo_all[((elo_grass['Winner']==i.Winner)&(elo_all['Loser']==i.Loser))|((elo_all['Loser']==i.Winner)&(elo_all['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Daniel Cukierman beat Alexey Vatutin
David Jorda Sanchis beat Giovanni Fonio


In [93]:
if final_hard is not None:
    final_hard=final_hard

else:
    final_hard=pd.DataFrame(columns=['Fav_Top100','Dog_Top100','Sex','Resulted','Time','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_hard[(final_hard['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,18:00,0.0,0.0,Li Tu,256.0,325.0,1.71,0.400000,0.00,0.00,0.2,Philip Sekulic,2.11,0.400000,0.00,0.00,0.4
1,18:00,0.0,0.0,Daniel Cukierman,386.0,512.0,1.70,0.300000,0.00,0.00,0.6,Alexey Vatutin,2.06,0.400000,0.56,0.11,0.4
2,18:00,0.0,0.0,Nick Chappell,356.0,417.0,1.74,0.700000,0.80,0.00,0.4,Hoang Ly Nam,2.05,0.166667,NaN,NaN,0.0
3,18:00,0.0,0.0,Ricardas Berankis,254.0,338.0,1.54,0.600000,0.75,0.16,0.2,Yasutaka Uchiyama,2.41,0.363636,0.78,0.00,0.2
4,19:30,0.0,0.0,Ugo Blanchet,320.0,630.0,1.33,0.400000,0.00,0.00,0.4,Mark Whitehouse,3.08,0.333333,0.00,0.00,0.4
5,21:00,1.0,0.0,Gastao Elias,358.0,560.0,1.43,0.636364,0.00,0.00,0.6,Miguel Damas,2.66,0.333333,NaN,NaN,0.4
6,21:00,0.0,0.0,Damir Dzumhur,211.0,446.0,1.55,0.700000,0.67,0.24,0.6,Daniel Cox,2.33,0.555556,0.00,0.00,0.4
7,22:30,1.0,0.0,Bernard Tomic,292.0,439.0,1.65,0.600000,0.00,0.00,0.4,Adria Soriano Barrera,2.03,0.571429,0.00,0.00,0.4
8,22:30,0.0,0.0,Charles Broom,359.0,531.0,1.67,0.600000,0.00,0.00,0.4,Edas Butvilas,2.09,0.500000,NaN,NaN,0.6


In [94]:
if final_clay is not None:
    final_clay=final_clay

else:
    final_clay=pd.DataFrame(columns=['Fav_Top100','Dog_Top100','Sex','Resulted','Time','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_clay[(final_clay['Sex']=='Womens')&(final_clay['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc


In [95]:
final_clay[(final_clay['Sex']=='Mens')&(final_clay['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,18:00,0.0,0.0,Manuel Guinard,262.0,331.0,1.63,0.800000,0.69,0.26,0.6,Matteo Martineau,2.18,0.400000,0.0,0.0,0.6
1,18:00,0.0,0.0,Calvin Hemery,300.0,696.0,1.29,0.600000,0.00,0.00,0.6,Luca Castagnola,3.40,0.200000,NaN,NaN,0.2
2,18:00,0.0,0.0,Fa Rodriguez Taverna Santiago,270.0,388.0,1.52,0.636364,NaN,NaN,0.2,Karl Friberg,2.42,0.200000,0.0,0.0,0.2
3,19:30,0.5,0.3,Enrico Dalla Valle,340.0,372.0,1.55,0.600000,NaN,NaN,0.6,Miljan Zekic,2.30,0.181818,0.0,0.0,0.6
4,19:30,0.0,0.0,Kyrian Jacquet,306.0,385.0,1.42,0.900000,0.00,0.00,0.6,Samuel Vincent Ruggeri,2.73,0.285714,0.0,0.0,0.4
5,19:30,0.0,0.0,Alexander Weis,295.0,452.0,1.33,0.545455,0.00,0.00,0.6,Lorenzo Rottoli,3.15,0.200000,NaN,NaN,0.2
6,21:00,0.0,0.0,Mathias Bourgue,279.0,731.0,1.53,0.600000,0.00,0.00,0.6,Andrea Arnaboldi,2.40,0.454545,0.0,0.0,0.4
7,21:00,1.0,0.0,Moez Echargui,290.0,354.0,1.56,0.600000,0.00,0.00,0.6,Maxime Chazal,2.31,0.375000,0.0,0.0,0.4
8,22:30,0.0,0.0,Giovanni Fonio,289.0,373.0,1.36,0.636364,0.00,0.00,0.4,David Jorda Sanchis,2.97,0.500000,0.0,0.0,0.6
9,22:30,0.0,0.0,Chun Hsin Tseng,357.0,333.0,1.58,0.500000,NaN,NaN,0.6,Louis Wessels,2.26,0.400000,0.0,0.0,0.4


In [96]:
if final_hard is not None:
    final_hard.to_pickle('Hard_Today')
else:
    final_clay[final_clay['Sex']=='k'].to_pickle('Hard_Today')
if final_clay is not None:
    final_clay.to_pickle('Clay_Today')  
final_all.to_pickle('All_Today')    
final_clay=final_clay[(final_clay['Resulted']=='False')].copy()
final_hard=final_hard[(final_hard['Resulted']=='False')].copy()    

In [97]:
final_all[(final_all['Sex']=='Mens')&(final_all['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,18:00,0.0,0.0,Ricardas Berankis,254.0,338.0,1.54,0.636364,0.75,0.16,0.4,Yasutaka Uchiyama,2.41,0.363636,0.78,0.00,0.2
1,18:00,0.0,0.0,Calvin Hemery,300.0,696.0,1.29,0.700000,0.00,0.00,0.6,Luca Castagnola,3.40,0.200000,NaN,NaN,0.2
2,18:00,0.3,0.0,Manuel Guinard,262.0,331.0,1.63,0.789474,0.69,0.26,0.6,Matteo Martineau,2.18,0.272727,0.00,0.00,0.6
3,18:00,0.5,0.0,Li Tu,256.0,325.0,1.71,0.400000,0.00,0.00,0.6,Philip Sekulic,2.11,0.400000,0.00,0.00,0.6
4,18:00,0.0,0.0,Nick Chappell,356.0,417.0,1.74,0.600000,0.80,0.00,0.4,Hoang Ly Nam,2.05,0.166667,NaN,NaN,0.0
5,18:00,0.0,0.0,Daniel Cukierman,386.0,512.0,1.70,0.300000,0.00,0.00,0.6,Alexey Vatutin,2.06,0.500000,0.56,0.11,0.4
6,18:00,0.0,0.0,Fa Rodriguez Taverna Santiago,270.0,388.0,1.52,0.500000,NaN,NaN,0.2,Karl Friberg,2.42,0.400000,0.00,0.00,0.2
7,19:30,0.0,0.0,Kyrian Jacquet,306.0,385.0,1.42,0.500000,0.00,0.00,0.6,Samuel Vincent Ruggeri,2.73,0.250000,0.00,0.00,0.4
8,19:30,0.0,0.0,Evan Zhu,371.0,419.0,1.74,0.500000,0.00,0.00,0.2,Stefanos Sakellaridis,2.05,0.500000,NaN,NaN,0.6
9,19:30,0.5,0.2,Enrico Dalla Valle,340.0,372.0,1.55,0.600000,NaN,NaN,0.6,Miljan Zekic,2.30,0.200000,0.00,0.00,0.6


In [98]:
final_all[(final_all['Sex']=='Womens')&(final_all['Resulted']=='False')][['Time','Fav_Top100','Dog_Top100','Fav','fav_rank','dog_rank','Fav_Odds','fav_percent','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav_Top100,Dog_Top100,Fav,fav_rank,dog_rank,Fav_Odds,fav_percent,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
